# Warehouse Location Problem

BBasket is an Indian corporation which operates a chain of membership-only grocery clubs. Currently they have thousands of customers in the following locations: Bangalore, Chennai, Hyderabad and Pune. They want to open two warehouses from the given list of locations available to them : Anantapur, Mysore and Vellore. Following is the distance given between warehouse and the customer city. As an analyst, provide them a solution to have minimum total cost of serving all the customers. A customer location can be served by only 1 active warehouse. The table below shows distances between cities in kilometres.

| W / C | Hyderabad | Chennai | Bangalore | Pune |
| :---- | :---- | :---- | :---- | :---- |
|Anantpur|422|482|215|882|
|Mysore|797|482|144|934|
|Vellore|779|157|201|1138|


##  Mathematical formulation / Pyomo components

---

**Sets:** 

To define the indexes for the given problem <br>
- Warehouse location, $w \in W$
- Customer location, $c \in C$

---

**Parameters:**

- Distance between each pair of warehouse-customer location - $\text {d}_{w,c}$ <br>
- Maximum number of warehouses that can be opened - P

---

**Decision variables:**


- Binary/Boolean variable for warehouse supplying to a customer location <br>

> ${x}_{w,c} \ \ where \ \ 0 \leq x_{w,c} \leq 1 \,\,\,\,\,\,\,\,\, \forall c \in C, w \in W $<br>

- Binary/Boolean variable for active warehouse location <br>

> ${y}_{w} \ \ where \ \ y_{w} \in \{0,1\} \,\,\,\,\,\,\,\,\, \forall w \in W $<br>


---


**Objective:**

To *minimize* the sum of distances between the warehouses and customer locations
    
\begin{align}
\textrm {min}\sum \limits _{w,c} \text {d}_{w,c} * \text {x}_{w,c}
\end{align}

---

**Constraints:**

*   One customer location can be satisfied by only one warehouse <br>
>$\sum_{w \in W} x_{w,c} = 1 \,\,\,\,\,\,\,\,\, \forall c \in C$ <br> 

*   Customer location can get goods from only active warehouse <br>

 > $x_{w,c} \leq y_{w} \,\,\,\,\,\,\,\,\, \forall c \in C, w \in W$ <br>

*   Active warehouses should be less than or equal to the required number of warehouses <br>
>$\sum_{w \in W} y_{w} <= P $



### Step1:
<b> Import Pyomo Environment </b>

In [ ]:
#Importing Pyomo environment and other required libraries

from pyomo.environ import *

import pandas as pd

### Step2:
<b>Specify / import data</b>

**Sets:** 

To define the indexes for the given problem <br>
- Warehouse location, $w \in W$
- Customer location, $c \in C$

In [ ]:
# Indexes - Defining sets/lists containing the indexes for this problem. This is a two index problem - warehouse location 
# and customer location will be the indexes




**Parameters:**

- Distance between each pair of warehouse-customer location - $\text {d}_{w,c}$ <br>
- Maximum number of warehouses that can be opened - P

In [ ]:
# parameters - distance between each warehouse-customer location

# Maximum number of warehouses that are allowed to be open


### Step3:
<b> Create Model Object</b> <br>

In [ ]:
# Creating an instance of a Concrete model since we have all the required data before hand



### Step4 

<b>Define Decision Variable</b>:


- Binary/Boolean variable for warehouse supplying to a customer location <br>

> ${x}_{w,c} \ \ where \ \ 0 \leq x_{w,c} \leq 1 \,\,\,\,\,\,\,\,\, \forall c \in C, w \in W $<br>

- Binary/Boolean variable for active warehouse location <br>

> ${y}_{w} \ \ where \ \ y_{w} \in \{0,1\} \,\,\,\,\,\,\,\,\, \forall w \in W $<br>

In [ ]:
#Define variables


### Step5:
<b>Define Objective</b>


To *minimize* the sum of distances between the warehouses and customer locations
    
\begin{align}
\textrm {min}\sum \limits _{w,c} \text {d}_{w,c} * \text {x}_{w,c}
\end{align}

In [ ]:
# Minimize the distance between warehouse and customer location



### Step6:
<b>Define Constraints</b>

*   One customer location can be satisfied by only one warehouse <br>
>$\sum_{w \in W} x_{w,c} = 1 \,\,\,\,\,\,\,\,\, \forall c \in C$ <br> 


In [ ]:
# single warehouse per customer



*   Customer location can get goods from only active warehouse <br>

 > $x_{w,c} \leq y_{w} \,\,\,\,\,\,\,\,\, \forall c \in C, w \in W$ <br>


In [ ]:
#only active warehouse can serve a customer location



*   Active warehouses should be less than or equal to the required number of warehouses <br>
>$\sum_{w \in W} y_{w} <= P $

In [ ]:
# number of warehouses can't be more than what is required



### Step7:
<b>Create solver & solve model </b><br>

### Step8:
<b>Display Results </b>